#Import libraries

In [ ]:
!pip install array_to_latex

In [ ]:
import array_to_latex as a2l
import numpy as np
from IPython.display import Markdown, display
from jax import vmap

def ltx(a, fmt="{:6.5f}"):
  return a2l.to_ltx(np.array(a) , frmt=fmt, print_out=False)

#Probability (coins and bell problem)

We toss two coins and a bell rang when the coins have same face.

we want to know Probability of every states.

### Functions

In [ ]:
def P_a(a): #p(A)
  p=.5
  return a*p + (~a) * (1-p)

def P_b(b): #p(B)
  p=.5
  return b*p + (~b) *(1-p)

def P_c_ab(c,a,b): #p(C|A,B)
  return c == (a==b)

def P_abc(a,b,c): #p(A,B,C)
  return P_a(a) * P_b(b) * P_c_ab(c,a,b)

values = np.asarray([0, 1],dtype=bool) # is tail or head?
p = vmap(
    vmap(vmap(P_abc, in_axes=(None,None,0)),in_axes=(None,0,None)),
    in_axes=(0,None,None),)(values,values,values)

### Margins

In [ ]:
#notation
A = 0
B = 1
C = 2

p_AB = p.sum(axis=C)
p_AC = p.sum(axis=B)
p_BC = p.sum(axis=A)

display(
    Markdown(
        "$$p(A,B) = " + ltx(p_AB,"{:6.5f}") + ", \qquad " +
        "p(A,C) = " + ltx(p_AC,"{:6.5f}") + ", \qquad " +
        "p(B,C) = " + ltx(p_BC,"{:6.5f}") + "$$"
    )
)


p_A = p.sum(axis=(B,C))
p_B = p.sum(axis=(A,C))
p_C = p.sum(axis=(A,B))

display(
    Markdown(
        "$$p(A) = " + ltx(p_A,"{:4.2f}") + ", \qquad " +
        "p(B) = " + ltx(p_B,"{:4.2f}") + ", \qquad " +
        "p(C) = " + ltx(p_C,"{:4.2f}") + "$$"
    )
)

p_AB_C = (p / p.sum(axis=(A,B) , keepdims= True))[:,:,1]
p_AC_B = (p / p.sum(axis=(A,C) , keepdims= True))[:,1,:]
p_BC_A = (p / p.sum(axis=(B,C) , keepdims= True))[1,:,:]

display(
    Markdown(
        "$$p(A,B\mid C) = " + ltx(p_AB_C,"{:4.2f}") + ", \qquad " +
        "p(A,C\mid B) = " + ltx(p_AC_B,"{:4.2f}") + ", \qquad " +
        "p(B,C\mid A) = " + ltx(p_BC_A,"{:4.2f}") + "$$"
    )
)

$$p(A,B) = \begin{bmatrix}
  0.25000 &  0.25000\\
  0.25000 &  0.25000
\end{bmatrix}, \qquad p(A,C) = \begin{bmatrix}
  0.25000 &  0.25000\\
  0.25000 &  0.25000
\end{bmatrix}, \qquad p(B,C) = \begin{bmatrix}
  0.25000 &  0.25000\\
  0.25000 &  0.25000
\end{bmatrix}$$

$$p(A) = \begin{bmatrix}
  0.50 &  0.50
\end{bmatrix}, \qquad p(B) = \begin{bmatrix}
  0.50 &  0.50
\end{bmatrix}, \qquad p(C) = \begin{bmatrix}
  0.50 &  0.50
\end{bmatrix}$$

$$p(A,B\mid C) = \begin{bmatrix}
  0.50 &  0.00\\
  0.00 &  0.50
\end{bmatrix}, \qquad p(A,C\mid B) = \begin{bmatrix}
  0.50 &  0.00\\
  0.00 &  0.50
\end{bmatrix}, \qquad p(B,C\mid A) = \begin{bmatrix}
  0.50 &  0.00\\
  0.00 &  0.50
\end{bmatrix}$$

# probability ( burglar/alarm problem)

<img src='https://drive.google.com/uc?id=1rpGR5JC7spJlSIvO07Xmte4iaubic4CF'>

###intialize values

In [ ]:
p_break_in = 1e-3
p_earthquake = 1e-3
p_false_alarm = 1e-3
TPR_alarm = 0.99 #probabilty of alarm given break-in
EQ_alarm = 0.01 #probabilty of alarm going of if there's an earthquake

###function

In [ ]:
def P_b(b): #P(B) probability of a break_in
  p = p_break_in
  return b * p + (~b) * (1-p)

def P_e(e): #P(E) probability of an earthquake
  p = p_earthquake
  return e * p + (~e) * (1-p)

def P_a_eb(a,e,b): #P(A |E,B) probability of alarm given break-in and earthquake
  f = p_false_alarm
  a_b = TPR_alarm
  a_e = EQ_alarm

  a0 =  ((1-f)
          *(1.0 * (b==0)+(1.0 - a_b) * (b==1))
          *(1.0 * (e==0)+(1.0 - a_e) * (e==1)))

  return (a==0) * a0 + (a==1) * (1-a0)

def P_aeb(a,e,b): #P(A,E,B)
  return P_b(b) * P_e(e) * P_a_eb(a,e,b)

###Margins

In [ ]:
#Notation
A = 0
E = 1
B = 2

values = np.asarray([0, 1],dtype=bool)

p = vmap(vmap(vmap(P_aeb , in_axes=(None,None,0)),in_axes=(None,0,None)),
         in_axes=(0,None,None))(values,values,values)

p_A = p.sum(axis=(E,B))[1]

display(
    Markdown(f"The marginal probability of an alarm is $p(A=1)={p_A:.3f}$")
)

p_EB_A = (p/p.sum(axis=(E,B),keepdims=True))[1,:,:] #P(E,B|A=1)
p_B_A = p_EB_A.sum(axis=0)[1]
p_E_A = p_EB_A.sum(axis=1)[1]

p_B_EA = p_EB_A[1,1]/p_EB_A[1,:].sum() #P(B=1 | E=1 , A=1)

display(
    Markdown("The posterior for Break-in or earthquake given an alarm is $$p(B,E\mid A) = "+ltx(p_EB_A)+
             ",$$" + "with marginal conditionals given by $$p(B\mid A) = "+f"{p_B_A:.3f}"+
             "\\quad \\text{and} \\quad P(E\mid A) ="+ f"{p_E_A:.4f}" + ".$$" +
             "The marginal given alarm *and* radio announcement is $p(B=1 \mid E=1,A=1) = "+
             f"{p_B_EA:.2f}" + "$.")
)

The marginal probability of an alarm is $p(A=1)=0.002$

The posterior for Break-in or earthquake given an alarm is $$p(B,E\mid A) = \begin{bmatrix}
  0.49925 &  0.49476\\
  0.00549 &  0.00050
\end{bmatrix},$$with marginal conditionals given by $$p(B\mid A) = 0.495\quad \text{and} \quad P(E\mid A) =0.0060.$$The marginal given alarm *and* radio announcement is $p(B=1 \mid E=1,A=1) = 0.08$.